QA2.json 更侧重于保单基础信息和常规操作，而 QA3.json 则在理赔和权益相关问题上提供了更详细的内容

In [ ]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"第{page}页", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "文档来源": "文档文件名",\n'
        '    "页码": "页码",\n'
        '    "文本片段": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data"
    output_file = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA3.json"
    qa_per_doc = 5
    api_key = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"
    base_url = "https://open.bigmodel.cn/api/paas/v4"
    model_name = "glm-4-0520"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第6页",
"文本片段": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"文档来源": "电子行政运作手册_保单行政_2023_10.md",
"页码": "第7页",
"文本片段": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["文档来源"] = filename
                        parsed["页码"] = page
                        parsed["文本片段"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

<>:108: SyntaxWarning: invalid escape sequence '\d'
<>:109: SyntaxWarning: invalid escape sequence '\d'
<>:108: SyntaxWarning: invalid escape sequence '\d'
<>:109: SyntaxWarning: invalid escape sequence '\d'
C:\Users\吴芷萱\AppData\Local\Temp\ipykernel_10180\990077496.py:108: SyntaxWarning: invalid escape sequence '\d'
  input_folder = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data"
C:\Users\吴芷萱\AppData\Local\Temp\ipykernel_10180\990077496.py:109: SyntaxWarning: invalid escape sequence '\d'
  output_file = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA3.json"


重试 1/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第2段（页码：第22页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
重试 2/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第2段（页码：第22页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第2段（页码：第22页）：达到最大重试次数，无法获取有效数据
重试 1/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第3段（页码：第38页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
重试 2/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第3段（页码：第38页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第3段（页码：第38页）：达到最大重试次数，无法获取有效数据
重试 1/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第4段（页码：第56页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
重试 2/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第4段（页码：第56页）：Error code: 429 - {'error': {'code': '1113', 'message': '您的账户已欠费，请充值后重试。'}}
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第4段（页码：第56页）：达到最大重试次数，无法获取有效数据
重试 1/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第5段（页码：第72页）：Er

In [11]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(1)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(1).json"
    qa_per_doc = 35
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 6 column 28 (char 300)
重试 1/2 - 文件 电子行政运作手册_保单行政_2023_10.md 第7段（页码：19）：无法解析 JSON

成功保存 35 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(1).json


In [12]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(2)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(2).json"
    qa_per_doc = 35
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


成功保存 35 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(2).json


In [13]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(3)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(3).json"
    qa_per_doc = 30
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


成功保存 30 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(3).json


In [14]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(4)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(4).json"
    qa_per_doc = 30
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


成功保存 30 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(4).json


In [15]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(5)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(5).json"
    qa_per_doc = 32
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


成功保存 32 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(5).json


In [16]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(6)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(6).json"
    qa_per_doc = 30
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第22段（页码：17）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第22段（页码：17）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第22段（页码：17）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Extra data: line 1 column 18 (char 17)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第24段（页码：17）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 18 (char 17)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第24段（页码：17）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第24段（页码：17）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第29段（页码：18）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第29段（页码：18）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第29段（页码：18）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 1/2 - 文件 活耀人生危疾保-产品手册.md 第30段（页码：18）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 2/2 - 文件 活耀人生危疾保-产品手册.md 第30段（页码：18）：无法解析 JSON
错误 - 文件 活耀人生危疾保-产品手册.md 第30段（页码：18）：达到

In [17]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "请以严格的 JSON 格式输出，包含以下字段：question, answer, 文档来源, 页码, 文本片段。\n"
        "请参考以下两个示例了解输出格式与内容风格：\n\n"
        "【示例1】\n" + few_shot_examples[0] + "\n\n"
        "【示例2】\n" + few_shot_examples[1] + "\n\n"
        "【文档内容】\n" + text_segment + "\n\n"
        "请生成一个高质量的问答对（问题+答案）。\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "请严格按照以下 JSON 格式输出：\n"
        "{\n"
        '    "question": "问题内容",\n'
        '    "answer": "答案内容",\n'
        '    "source_file": "文档文件名",\n'
        '    "page_num": "页码",\n'
        '    "content": "原始文本段"\n'
        "}"
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(7).json"
    qa_per_doc = 35
    api_key = "8489cfb6-9837-443a-8fa6-808c1c040428"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-v3-250324"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第6段（页码：8）：无法解析 JSON
JSON 解析错误: Extra data: line 1 column 14 (char 13)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第6段（页码：8）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第6段（页码：8）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 6 column 150 (char 411)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第34段（页码：40）：无法解析 JSON

成功保存 34 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA5(7).json
